In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/1.prepare_data/1.seurat_pipeline.240507/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/3.Genome_wide_prediction/SCENT/SCENT.240520/"

In [3]:
n.cores = 64

Import candidate E-G pairs

In [4]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [5]:
matrix.atac = read.csv(path.matrix.atac_count,
                       row.names = 1,
                       check.names = F)
matrix.atac = Matrix(as.matrix(matrix.atac), sparse = TRUE)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

In [7]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]

Create SCENT object

In [8]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = "K562")
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
K562_Xu_AAACAGCCAAGCGATG-1,K562_Xu_AAACAGCCAAGCGATG-1,11087,1001,K562,9.313529,0.09028592
K562_Xu_AAACAGCCACATGCTA-1,K562_Xu_AAACAGCCACATGCTA-1,17288,2295,K562,9.757768,0.13275104
K562_Xu_AAACAGCCACGGTACT-1,K562_Xu_AAACAGCCACGGTACT-1,18989,1411,K562,9.851615,0.07430618
K562_Xu_AAACAGCCATAAAGCA-1,K562_Xu_AAACAGCCATAAAGCA-1,10270,1109,K562,9.236982,0.10798442
K562_Xu_AAACATGCAACTCGCG-1,K562_Xu_AAACATGCAACTCGCG-1,10962,1785,K562,9.302190,0.16283525
K562_Xu_AAACATGCAATCCTAG-1,K562_Xu_AAACATGCAATCCTAG-1,22686,2486,K562,10.029503,0.10958300
K562_Xu_AAACATGCACATAACT-1,K562_Xu_AAACATGCACATAACT-1,10649,512,K562,9.273221,0.04807963
K562_Xu_AAACATGCACCTATAG-1,K562_Xu_AAACATGCACCTATAG-1,15716,2278,K562,9.662435,0.14494782
K562_Xu_AAACATGCAGGGAGCT-1,K562_Xu_AAACATGCAGGGAGCT-1,12360,1570,K562,9.422221,0.12702265


In [9]:
gene_peak = as.data.frame(mcols(pairs.E2G.filter)[,c("TargetGene","PeakName")])
gene_peak

TargetGene,PeakName
<chr>,<chr>
ACOT7,chr1-10001196-10001745
AGTRAP,chr1-10001196-10001745
C1orf167,chr1-10001196-10001745
CAMTA1,chr1-10001196-10001745
CASZ1,chr1-10001196-10001745
CENPS,chr1-10001196-10001745
CHD5,chr1-10001196-10001745
CLCN6,chr1-10001196-10001745
CLSTN1,chr1-10001196-10001745


In [10]:
list.gene_peak <- split(gene_peak, seq_len(n.cores))

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


In [11]:
list.obj.SCENT <- lapply(list.gene_peak, function(gene_peak.tmp) {
  obj.SCENT.tmp <- CreateSCENTObj(
    rna = matrix.rna[rownames(matrix.rna) %in% gene_peak.tmp$TargetGene,],
    atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
    meta.data = meta.data,
    peak.info = gene_peak.tmp,
    covariates = c("log.nUMI","percent.mito"), 
    celltypes = "celltype"
  )
})

In [12]:
rm(matrix.rna)
rm(matrix.atac)
rm(meta.data)
rm(gene_peak)
rm(list.gene_peak)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,21880548,1168.6,33620184,1795.6,33620184,1795.6
Vcells,10856837568,82831.1,14336830217,109381.4,11085022594,84572.1


RUN SCENT prediction

In [13]:
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
        if (1) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            bs = boot::boot(df2, assoc, R = 100, formula = formula, 
                stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            if (p0 < 0.1 & bs$t0[1] > 0) {
                bs = boot::boot(df2, assoc, R = 500, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
            #     bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        }
    }
    object@SCENT.result <- res
    return(object)
}

In [ ]:
start_time <- Sys.time()
cl <- makeCluster(n.cores)
clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
list.SCENT.result <- parLapply(cl, list.obj.SCENT, function(obj.SCENT.tmp) {
  obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
  return(obj.SCENT.tmp@SCENT.result)
})
stopCluster(cl)
end_time <- Sys.time()
execution_time <- end_time - start_time

In [ ]:
execution_time

In [ ]:
list.SCENT.result[[1]]

Save results

In [ ]:
dir.create(dir.output,recursive = T)
saveRDS(list.SCENT.result,
        paste(dir.output,"list.SCENT.result.rds",sep = "/"))

In [ ]:
df.SCENT.result = do.call(rbind,list.SCENT.result)
rownames(df.SCENT.result) = paste(df.SCENT.result$peak,
                                  df.SCENT.result$gene,
                                  sep = "_")
df.SCENT.result

In [ ]:
names(pairs.E2G.filter2) = pairs.E2G.filter2$PairName
pairs.E2G.res = pairs.E2G.filter2[rownames(df.SCENT.result)]
mcols(pairs.E2G.res)[,c("beta","se","z","p","boot_basic_p")] = df.SCENT.result[,c("beta","se","z","p","boot_basic_p")]
pairs.E2G.res$boot_basic_p.log10 = -log10(pairs.E2G.res$boot_basic_p)
pairs.E2G.res$sign = 1
pairs.E2G.res$sign[pairs.E2G.res$beta < 0] = -1
pairs.E2G.res$boot_basic_p.log10.signed = pairs.E2G.res$boot_basic_p.log10 * pairs.E2G.res$sign
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

In [ ]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

In [ ]:
sessionInfo()